<a href="https://colab.research.google.com/github/emmayeh-0211/2021AI_dl/blob/main/05_Data_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download dataset from https://drive.google.com/file/d/12iinuMZkaZOZGXlkPsUnmETaE-cMlyHu/view?usp=sharing
!gdown --id 12iinuMZkaZOZGXlkPsUnmETaE-cMlyHu --output defect.zip

Downloading...
From: https://drive.google.com/uc?id=12iinuMZkaZOZGXlkPsUnmETaE-cMlyHu
To: /content/defect.zip
53.2MB [00:00, 61.4MB/s]


In [ ]:
# unzip file
!unzip -q defect.zip

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
from glob import glob # 搜尋路徑的套件
from sklearn.model_selection import train_test_split # sklearn中 切成training data 跟 test data 的套件功能

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, img_paths, batch_size, img_size, shuffle=True, aug=False):
        self.img_paths = img_paths
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.aug = aug

        self.indexes = np.arange(len(self.img_paths)) # 假設有6筆資料，產生的np array就是[0, 1, 2, 3, 4, 5]
        self.on_epoch_end()

    def __len__(self): # 每一個epoch在產生資料的時候，總共有幾個batch
        return int(np.ceil(len(self.img_paths) / self.batch_size)) # batches per epoch
          # 資料筆數 / batch size，其實就是在計算一個epoch有幾個batches

    def __getitem__(self, index): # 要拿第幾個batch
        # Generate indexes of the batch
        idxs = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # Find list of IDs
        batch_img_paths = [self.img_paths[i] for i in idxs]

        # Generates data containing batch_size samples
        x = np.empty((len(batch_img_paths), self.img_size, self.img_size, 3), dtype=np.float32) # x代表有幾張圖
        y = np.empty((len(batch_img_paths), 1), dtype=np.float32) # y代表就是label，如果是分類，就是分類向量的內容

        for i, img_path in enumerate(batch_img_paths):
            # read img
            img = cv2.imread(img_path)
            img = cv2.resize(img, (self.img_size, self.img_size))
            img = img / 255.

            if self.aug:
                img = img # Augmentation
                
            # read class label
            cls = img_path.split('/')[-3]
            if cls == 'MT_Free':
                cls = 0
            else:
                cls = 1
            x[i] = img
            y[i] = cls
            # 如果是做多分類的話，在return出去之前要先做one-hot encoding
        return x, y

    def on_epoch_end(self):
        # Updates indexes after each epoch
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [ ]:
img_paths = glob('./Magnetic-tile-defect/MT_Free/Imgs/*.jpg') + glob('./Magnetic-tile-defect/MT_Blowhole/Imgs/*.jpg')
# *代表符合.jpg的所有檔案

In [ ]:
len(img_paths)

1067

In [ ]:
train_img_paths, val_img_paths = train_test_split(img_paths, test_size=0.2)
# 測試資料留0.2，因此訓練資料會是0.8

In [ ]:
len(train_img_paths), len(val_img_paths)

(853, 214)

In [ ]:
train_gen = DataGenerator(train_img_paths, 32, 200, shuffle=True, aug=True)
val_gen = DataGenerator(val_img_paths, 32, 200, shuffle=False, aug=False)

In [ ]:

# get 1 batch
imgs, labels = train_gen[2]
# imgs 圖片 labels 正確答案

In [ ]:
imgs.shape, labels.shape

((32, 200, 200, 3), (32, 1))

In [ ]:
print(labels)

In [ ]:
# train

model.fit(train_gen,
          validation_data=val_gen,
          epochs=10,
          )